# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)


In [35]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

다음은 이메일 본문 예시입니다.


In [37]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

출력 파서를 사용하지 않는 경우 예시

In [38]:
def stream_response(response):
    answer = ''
    for token in response:
        if isinstance(token, AIMessageChunk):
            answer += token.content
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            answer += token
            print(token, end="", flush=True)

    return answer

In [39]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessageChunk

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = prompt | llm

response = chain.stream({"email_conversation": email_conversation})

answer = ''
for token in response:
    if isinstance(token, AIMessageChunk):
        answer += token.content
        print(token.content, end="", flush=True)
    elif isinstance(token, str):
        answer += token
        print(token, end="", flush=True)

- "ZENESIS" 자전거에 대한 상세한 정보 요청
- 유통 전략과 마케팅 계획을 구체화하기 위한 협력 제안
- 다음 주 화요일(1월 15일) 오전 10시에 미팅 제안

위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱해 보겠습니다.

참고로, Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명입니다. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다. 그러므로 이 설명은 정확하고 명확해야 합니다.

In [40]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [41]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [42]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [43]:
# chain 을 생성합니다.
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [44]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response)

{
  "properties": {
    "person": {
      "description": "메일을 보낸 사람",
      "title": "Person",
      "type": "string"
    },
    "email": {
      "description": "메일을 보낸 사람의 이메일 주소",
      "title": "Email",
      "type": "string"
    },
    "subject": {
      "description": "메일 제목",
      "title": "Subject",
      "type": "string"
    },
    "summary": {
      "description": "메일 본문을 요약한 텍스트",
      "title": "Summary",
      "type": "string"
    },
    "date": {
      "description": "메일 본문에 언급된 미팅 날짜와 시간",
      "title": "Date",
      "type": "string"
    }
  },
  "required": ["person", "email", "subject", "summary", "date"]
}

마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [48]:
output

'{\n  "properties": {\n    "person": {\n      "description": "메일을 보낸 사람",\n      "title": "Person",\n      "type": "string"\n    },\n    "email": {\n      "description": "메일을 보낸 사람의 이메일 주소",\n      "title": "Email",\n      "type": "string"\n    },\n    "subject": {\n      "description": "메일 제목",\n      "title": "Subject",\n      "type": "string"\n    },\n    "summary": {\n      "description": "메일 본문을 요약한 텍스트",\n      "title": "Summary",\n      "type": "string"\n    },\n    "date": {\n      "description": "메일 본문에 언급된 미팅 날짜와 시간",\n      "title": "Date",\n      "type": "string"\n    }\n  },\n  "required": ["person", "email", "subject", "summary", "date"]\n}'

In [52]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.

structured_output = parser.parse(output)
print(structured_output)

OutputParserException: Failed to parse EmailSummary from completion {"properties": {"person": {"description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "title": "Person", "type": "string"}, "email": {"description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "title": "Email", "type": "string"}, "subject": {"description": "\uba54\uc77c \uc81c\ubaa9", "title": "Subject", "type": "string"}, "summary": {"description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "title": "Summary", "type": "string"}, "date": {"description": "\uba54\uc77c \ubcf8\ubb38\uc5d0 \uc5b8\uae09\ub41c \ubbf8\ud305 \ub0a0\uc9dc\uc640 \uc2dc\uac04", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}. Got: 5 validation errors for EmailSummary
person
  Field required [type=missing, input_value={'properties': {'person':...ct', 'summary', 'date']}, input_type=dict]
    For further information visit 

## parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [53]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [54]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

OutputParserException: Failed to parse EmailSummary from completion {"properties": {"person": {"description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "title": "Person", "type": "string"}, "email": {"description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "title": "Email", "type": "string"}, "subject": {"description": "\uba54\uc77c \uc81c\ubaa9", "title": "Subject", "type": "string"}, "summary": {"description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "title": "Summary", "type": "string"}, "date": {"description": "\uba54\uc77c \ubcf8\ubb38\uc5d0 \uc5b8\uae09\ub41c \ubbf8\ud305 \ub0a0\uc9dc\uc640 \uc2dc\uac04", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}. Got: 5 validation errors for EmailSummary
person
  Field required [type=missing, input_value={'properties': {'person':...ct', 'summary', 'date']}, input_type=dict]
    For further information visit 

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [16]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [ ]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.